In [1]:
#import Libraries
import tensorflow as tf
import numpy as np



In [2]:
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import Adam
import LoadDataset as LD


batch_size = 128
num_classes = 10
epochs = 10

# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

Using TensorFlow backend.


In [112]:
#data augmentation
#Image translation
def flip(image, classlabel):
    #image is np array shape[width, height, channel]
    im = image
    y = np.array([classlabel])
    shape = [np.shape(image)[0], np.shape(image)[1], np.shape(image)[2]]
    x = tf.placeholder(dtype = tf.float16, shape = shape)
    flip_1 = tf.image.flip_up_down(x)
    flip_2 = tf.image.flip_left_right(x)
    flip_3 = tf.image.random_flip_up_down(x)
    flip_4 = tf.image.random_flip_left_right(x)
    with tf.Session() as sess:
        aug_x = sess.run([flip_1,flip_2,flip_3,flip_4], feed_dict={x:im })
    y = np.array([classlabel,classlabel,classlabel,classlabel])

    return aug_x, y 


def flip_augmentation(images, labels):
    X = images
    print(np.shape(X))
    y =  labels
    for i in range(np.shape(labels)[0]):
        X = np.concatenate([X,flip(images[i],labels[i])[0]])
        y = np.concatenate([y,flip(images[i],labels[i])[1]]) 
    return X,y

[x_train, y_train] = flip_augmentation(x_train[0:2],y_train[0:2])
print(x_train)
print(y_train)

#PCA

(2, 32, 32, 3)
[[[[ 59.  62.  63.]
   [ 43.  46.  45.]
   [ 50.  48.  43.]
   ...
   [158. 132. 108.]
   [152. 125. 102.]
   [148. 124. 103.]]

  [[ 16.  20.  20.]
   [  0.   0.   0.]
   [ 18.   8.   0.]
   ...
   [123.  88.  55.]
   [119.  83.  50.]
   [122.  87.  57.]]

  [[ 25.  24.  21.]
   [ 16.   7.   0.]
   [ 49.  27.   8.]
   ...
   [118.  84.  50.]
   [120.  84.  50.]
   [109.  73.  42.]]

  ...

  [[208. 170.  96.]
   [201. 153.  34.]
   [198. 161.  26.]
   ...
   [160. 133.  70.]
   [ 56.  31.   7.]
   [ 53.  34.  20.]]

  [[180. 139.  96.]
   [173. 123.  42.]
   [186. 144.  30.]
   ...
   [184. 148.  94.]
   [ 97.  62.  34.]
   [ 83.  53.  34.]]

  [[177. 144. 116.]
   [168. 129.  94.]
   [179. 142.  87.]
   ...
   [216. 184. 140.]
   [151. 118.  84.]
   [123.  92.  72.]]]


 [[[154. 177. 187.]
   [126. 137. 136.]
   [105. 104.  95.]
   ...
   [ 91.  95.  71.]
   [ 87.  90.  71.]
   [ 79.  81.  70.]]

  [[140. 160. 169.]
   [145. 153. 154.]
   [125. 125. 118.]
   ...
   [ 9